In [1]:
import os
import numpy as np
import tensorflow as tf
import keras
from tensorflow.python.keras import Input, Model, Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Dropout, Dense, BatchNormalization, LSTM, Flatten
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K

In [2]:
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

dataset_path = os.path.join('dataset_result', 'no_preprocess.npz')


In [3]:
dataset = np.load(dataset_path)
features, labels = dataset['features'], dataset['labels']

labels = labels.reshape((-1, 1))
labels = OneHotEncoder().fit_transform(labels).toarray()

In [4]:
def lstm_model():
    model = Sequential([
        LSTM(360, input_shape=(14, 4000), activation=tf.nn.leaky_relu, return_sequences=True),
        Dropout(0.4),
        BatchNormalization(),
        LSTM(360, activation=tf.nn.leaky_relu, return_sequences=True),
        Dropout(0.3),
        BatchNormalization(),
        Flatten(),
        Dense(250, activation=tf.nn.leaky_relu),
        Dropout(0.2),
        Dense(2, activation='sigmoid', name='output_layer')
    ])

    return model

In [5]:
def run_network(model: keras.Model, train, val, test, iteration, epochs=30):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    # Get training, validation and testing data from parameters
    x_train, y_train = train[0], train[1]
    x_val, y_val = val[0], val[1]
    x_test, y_test = test[0], test[1]

    model.fit(
        x=x_train,y=y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)]
    ) # train the model

    eval = model.evaluate(x_test, y_test) # get results
    print('{}. Accuracy: '.format(iteration), eval[1] * 100)
    return eval[1]


In [6]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1)

num_splits = 10
iteration = 1

results = []

for train_idx, val_idx in KFold(n_splits=num_splits).split(x_train):
    x_train_curr, y_train_curr = x_train[train_idx], y_train[train_idx]
    x_val_curr, y_val_curr = x_train[val_idx], y_train[val_idx]

    model = lstm_model() # create a fresh lstm model

    result = run_network(model, (x_train_curr, y_train_curr), (x_val_curr, y_val_curr),
                             (x_test, y_test), epochs=30, iteration=iteration)

    results.append(result)
    K.clear_session() # clear tensorflow data
    del model # delete the object
    iteration += 1



Epoch 1/30
76/76 [==============================] - 16s 154ms/step - loss: 1.0831 - accuracy: 0.4901 - val_loss: 0.7743 - val_accuracy: 0.5224